# 1.2 Tools

The defining difference between a standard chat model and an agent is the access to tools. While a model can only "think," an agent can interact with external systems to perform tasks, which is the final step in the agent-building process. 

Most modern AI agents follow the ReAct pattern, where the agent uses tools to interact with the world. Tools transform a passive language model into an active coordinator capable of accessing data, executing tasks, or even calling other agents.

## 1.2.1 Tool definition

To create a tool in LangChain, you define a standard Python function and apply specific metadata so the agent can "see" it.

You must use the `@tool` decorator to convert a function into a tool the agent can use.

The agent relies on the tool's name and description to understand when and how to use it.  
    ◦ By default, these are pulled from the function name and its docstring, but you can override them within the decorator.  
    ◦ If a tool's description is clear, the agent should know how to use it automatically, though sometimes reinforcing this in a system prompt can help.

In [ ]:
from langchain.tools import tool

@tool
def square_root(x: float) -> float:
    """Calculate the square root of a number"""
    return x ** 0.5

In [ ]:
@tool("square_root")
def tool1(x: float) -> float:
    """Calculate the square root of a number"""
    return x ** 0.5

In [ ]:
@tool("square_root", description="Calculate the square root of a number")
def tool1(x: float) -> float:
    return x ** 0.5

In [ ]:
tool1.invoke({"x": 467})

## 1.2.2 Adding to agents

When an agent uses a tool, the communication flow involves more than just a simple question and answer. In a notebook, you can print individual messages to see this sequence:

1. Human Message: The initial user query.  
2. AI Message (Tool Call): The model returns an AI message where the text content might be empty, but it contains a tool call field with the tool name, arguments, and a unique ID.  
3. Tool Message: A specific message type that provides the output from the tool back to the agent.  
4. Final AI Message: The agent uses the tool's output to formulate a final response to the user.  

In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
import os
from langchain.agents import create_agent
from langchain_openai import AzureChatOpenAI

model = AzureChatOpenAI(
    azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
)
agent = create_agent(
    model=model,
    tools=[tool1],
    system_prompt="You are an arithmetic wizard. Use your tools to calculate the square root and square of any number."
)

In [ ]:
from langchain.messages import HumanMessage

question = HumanMessage(content="What is the square root of 467?")

response = agent.invoke(
    {"messages": [question]}
)

print(response['messages'][-1].content)

In [ ]:
from pprint import pprint

pprint(response['messages'])

In [ ]:
print(response["messages"][1].tool_calls)